# Docker

# Twint

In [7]:
# %cd /workspace/twint
# !pip install e .
# !twint -u CNBC

/workspace/twint/twint
Traceback (most recent call last):
  File "cli.py", line 15, in <module>
    from . import run
ImportError: attempted relative import with no known parent package
